In [ ]:
from jubilee_controller import JubileeMotionController
import matplotlib.pyplot as plt
import time
import numpy as np 
from pynput import keyboard
import cv2
from datetime import datetime



In [2]:
jubilee = JubileeMotionController()
jubilee.gcode('M98 P"/sys/config.g"')


'ok\n'

Duas células a seguir fazem o home dos eixos principais.

In [3]:
jubilee.home_all()

In [4]:
jubilee.move_xyz_relative(z=100)

Para de emergência e parada de drivers

In [ ]:
#Para fazer uma parada de emergência, cancele a última célula executada e rode esta.
jubilee.emergence_stop()

In [ ]:
jubilee.stop_drivers(['Z'])

Movimento relativo em relação às coordenadas atuais:

In [ ]:
jubilee.home_all_forced()

In [ ]:
jubilee.axes_homed

[True, True, True, True]

In [28]:
jubilee.move_xyz_relative(z=-100)

Exemplo de movimento absoluto e aleatório:

In [ ]:
for i in range(100):
    jubilee.move_xyz_absolute(x=50, y=150,velocity=18000)
    jubilee.move_xyz_absolute(x=200, y=150,velocity=18000)
    jubilee.move_xyz_absolute(x=200, y=50,velocity=18000)
    jubilee.move_xyz_absolute(x=50, y=50,velocity=18000)
    


KeyboardInterrupt: 

In [ ]:
for i in range(10000):
    x,y,z = np.random.randint(100,320),np.random.randint(100,320),np.random.randint(100,120)
    jubilee.move_xyz_absolute(x=x, y=y,velocity=2000)
    if np.random.choice([True,False],1,p=[0.2,0.8])[0]:
        jubilee.move_xyz_absolute(z=z)
    
    print(jubilee.position, i, end ='\r')
    time.sleep(np.random.randint(10,20))
    
    


KeyboardInterrupt: 

In [ ]:
jubilee.move_xyz_absolute(x=100, y=200)

In [ ]:
raio = 165  
centro_x = 165
centro_y = 0
passo = 2  

for x in range(-raio, raio + 1, passo):
    y = (raio**2 - x**2)**(1/2)
    x_pos = centro_x + x
    y_pos = centro_y + y
    jubilee.move_xyz_absolute(x_pos, y_pos)


Aqui é pra controlar com o teclado

In [16]:


step = 10
pressed_keys = set()
running = True

def on_press(key):
    global running
    try:
        if key.char in ['1', '2', '3', '4', '9', '0','5','6']:
            pressed_keys.add(key.char)
    except AttributeError:
        if key == keyboard.Key.esc:
            running = False

def on_release(key):
    try:
        if key.char in pressed_keys:
            pressed_keys.remove(key.char)
    except AttributeError:
        pass

listener = keyboard.Listener(on_press=on_press, on_release=on_release)
listener.start()

try:
    while running:
        if '1' in pressed_keys:
            jubilee.move_xyz_relative(0, step, 0)
        elif '2' in pressed_keys:
            jubilee.move_xyz_relative(0, -step, 0)
        elif '3' in pressed_keys:
            jubilee.move_xyz_relative(-step, 0, 0)
        elif '4' in pressed_keys:
            jubilee.move_xyz_relative(step, 0, 0)
        elif '9' in pressed_keys:
            jubilee.move_xyz_relative(0, 0, -step)
        elif '0' in pressed_keys:
            jubilee.move_xyz_relative(0, 0, step)
        elif '5' in pressed_keys:
            step+=1
        elif '6' in pressed_keys:
            step-=1
            if step<1:
                step=1
        time.sleep(0.005)
        print(jubilee.position, step, end ='\r')

except KeyboardInterrupt:
    print("Interrompido")

listener.stop()


Interrompido0, 250.0] 10


In [5]:

class camera_tool:
    def __init__(self, machine: JubileeMotionController = None):
        self.installed = False
        self.machine = machine
        self.move_velocity = 10000

    def install(self):
        self.machine.move_xyz_absolute(y=220, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=302, velocity=self.move_velocity)
        jubilee.gcode("G0 U70")
        self.machine.move_xyz_absolute(y=7, velocity=self.move_velocity)
        jubilee.gcode("G0 U0")
        self.machine.move_xyz_absolute(y=70, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=50, y=120, velocity=self.move_velocity)

    def uninstall(self):
        self.machine.move_xyz_absolute(y=90, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=302, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(y=7, velocity=self.move_velocity)
        jubilee.gcode("G0 U70")
        self.machine.move_xyz_absolute(y=70, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=50, y=120, velocity=self.move_velocity)
        jubilee.gcode("G0 U0")

    def photo(self, filename='captura.jpg'):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Erro ao abrir a câmera.")
            return

        ret, frame = cap.read()
        if not ret:
            print("Erro ao capturar a imagem.")
            cap.release()
            return

        cv2.imwrite(filename, frame)
        print(f"Foto salva como {filename}")

        cap.release()

    
import cv2
import numpy as np

class camera_tool:
    def __init__(self, machine=None):
        self.installed = False
        self.machine = machine
        self.move_velocity = 10000

    def install(self):
        self.machine.move_xyz_absolute(y=220, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=302, velocity=self.move_velocity)
        jubilee.gcode("G0 U70")
        self.machine.move_xyz_absolute(y=7, velocity=self.move_velocity)
        jubilee.gcode("G0 U0")
        self.machine.move_xyz_absolute(y=70, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=50, y=120, velocity=self.move_velocity)

    def uninstall(self):
        self.machine.move_xyz_absolute(y=90, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=302, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(y=7, velocity=self.move_velocity)
        jubilee.gcode("G0 U70")
        self.machine.move_xyz_absolute(y=70, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=50, y=120, velocity=self.move_velocity)
        jubilee.gcode("G0 U0")

    def photo(self, filename='captura.jpg',video_index=0):
        cap = cv2.VideoCapture(video_index)
        if not cap.isOpened():
            return

        ret, frame = cap.read()
        if not ret:
            cap.release()
            return

        cv2.imwrite(filename, frame)

        cap.release()



In [6]:
camera = camera_tool(jubilee)

In [24]:
camera.uninstall()

In [27]:
jubilee.move_xyz_relative(z=-50)

In [ ]:
x_inicial,y_inicial,z_inicial = 198.0, 196.0, 48.0
linhas = ['A','B','C','D']
colunas = [str(i) for i in range(1,7)]
for i in range(4):
    for j in range(6):
        jubilee.move_xyz_absolute(x=x_inicial-22*i,y=x_inicial+22*j,z=z_inicial,velocity=10000)
        time.sleep(5)
        camera.photo(f'fotos/{colunas[j]+linhas[i]}.jpg')

In [ ]:
for _ in range(2):
    camera.install()
    time.sleep(1)
    camera.uninstall()

In [ ]:
jubilee.stop_drivers(['U'])

In [ ]:
jubilee.gcode("M18")

''

In [ ]:
jubilee.position

[236.0, 284.0, 113.0]

In [9]:
posicao1 = [165.0, 211.0, 300.0]
posicao2 = [165.0, 316.0, 300.0]
posicoes = (posicao1,posicao2)

In [11]:
camera.photo(f'fa.jpg',video_index=0)

In [ ]:
n = 1000


for i in range(n):
    placa = ('placa 1','placa 2')
    for posicao,p in zip(posicoes,placa):
        now = datetime.now()

        current_time_hms = now.strftime("%H:%M:%S")
        x,y,z = posicao
        jubilee.move_xyz_absolute(x=x,y=y,z=z,velocity=8000)
        time.sleep(5)

        camera.photo(f'fotos2/{p} {current_time_hms}.jpg',video_index=1)
        print(i,end='\r')

    
    time.sleep(10)

[ WARN:0@9439.197] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@9439.198] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


KeyboardInterrupt: 